In [ ]:
#r "nuget:System.Text.Json, 8.0.1"
#r "nuget:Azure.Identity, 1.10.4"
#r "nuget:DotNetEnv, 3.0.0"

In [ ]:
using System.Net;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text.Json.Nodes;
using System.Text.Json;
using System.IO; 

using Azure;
using Azure.Core;
using Azure.Identity;
using DotNetEnv;

In [ ]:
Env.Load("../../.env");

string openAIEndpoint = Environment.GetEnvironmentVariable("COMPLETIONS_OPENAI_ENDPOINT");
string openAIModelDeployment = Environment.GetEnvironmentVariable("COMPLETIONS_OPENAI_COMPLETION_MODEL_DEPLOYMENT");
string openAIApiVersion = "2024-03-01-preview";

var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions { 
    ExcludeEnvironmentCredential = true,
    ExcludeManagedIdentityCredential = true,
    ExcludeSharedTokenCacheCredential = true,
    ExcludeInteractiveBrowserCredential = true,
    ExcludeAzurePowerShellCredential = true,
    ExcludeVisualStudioCodeCredential = false,
    ExcludeAzureCliCredential = false
});

var token = await credential.GetTokenAsync(new TokenRequestContext(new[] { "https://cognitiveservices.azure.com/.default" }));

In [ ]:
var jsonStructure = new {
    classification = "",
    text = ""
};

var jsonStructureList = new List<object>();
jsonStructureList.Add(jsonStructure);

var requestText = @"
Dear Team,

Thank you for attending our recent meeting on building an AI solution using Azure. I wanted to summarize the key points and action items discussed during the meeting:

1. Project Overview:
  - We discussed the need for an AI solution to enhance customer interactions.
  - The goal is to integrate intelligence into our communications across various channels.
2. Key Takeaways:
  - Azure Communication Services and Azure AI can be used together to automate and transform customer interactions.
  - Our solution will provide faster and informed responses across text-based bots, voice channels, email, and SMS.
  - Agents can leverage AI context to handle escalations effectively.
3. Action Items:
  - Set up a project team to explore Azure Communication Services and Azure AI.
  - Identify communication channels (e.g., website chat, phone, SMS) for integration.
  - Develop a proof-of-concept bot using ChatGPT and Azure OpenAI Service.
  - Create a roadmap for implementing the solution.
4. Next Steps:
  - Let's schedule a follow-up meeting next week to review progress and assign responsibilities.
  - Let's grab a coffee and have a chat.

Feel free to reach out if you have any questions or need further clarification. Looking forward to working together on this exciting project!

Best regards, 
James";

var requestPayload = new JsonObject
{
  { "max_tokens", 4096 },
  { "temperature", 0.1 },
  { "top_p", 0.1 },
  { "logprobs", true },
  { "top_logprobs", 3 },
  { "messages", new JsonArray
    {
      new JsonObject
      {
        { "role", "system" },
        { "content", $"You are an AI assistant that splits text. classifies it into categories, and returns the response as structured JSON objects using the following structure: {JsonSerializer.Serialize(jsonStructureList)}. Do not return as a code block." }
      },
      new JsonObject
      {
        { "role", "user" },
        { "content", "Classify the following text in the following categories: 'top_priority', 'high_priority', 'medium_priority', 'low_priority'." }
      },
      new JsonObject
      {
        { "role", "user" },
        { "content", requestText }
      }
    }
  }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true
});

string endpoint = $"{openAIEndpoint}openai/deployments/{openAIModelDeployment}/chat/completions?api-version={openAIApiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("Authorization", $"Bearer {token.Token}");
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var responseMessage = await httpClient.PostAsync(endpoint, stringContent);

    if (responseMessage.IsSuccessStatusCode)
    {
        using (var responseStream = await responseMessage.Content.ReadAsStreamAsync())
        {
            using (var streamReader = new StreamReader(responseStream))
            {
                var response = await streamReader.ReadToEndAsync();
                File.WriteAllText("OpenAICompletions-REST-Response.json", response);
                Console.WriteLine("OpenAICompletions-REST-Response.json has been created with the response from the OpenAI API.");
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {responseMessage}");
    }
}